In [1]:
import os
import csv
import math
import random
import operator
import numpy as np
import timeit

class multinomialNaiveBayes:
    
    def __init__(self, training_features, training_labels):
        start = timeit.default_timer()
        self.numOfMessages = len(training_labels)
        self.numOfFeatures = len(training_features[0])
        self.FeaturePerLabel_0 = [0]*self.numOfFeatures # given label 0, the feature x has been observed FeaturePerLabel_0[x] times
        self.FeaturePerLabel_1 = [0]*self.numOfFeatures # given label 1, the feature x has been observed FeaturePerLabel_1[x] times
        
        for featureNum in range(0,self.numOfFeatures):
            for row in range(0,len(training_labels)):
                if (training_labels[row] == 0):
                    self.FeaturePerLabel_0[featureNum] += training_features[row,featureNum]
                else:
                    self.FeaturePerLabel_1[featureNum] += training_features[row,featureNum]
        
        self.numOfSpams = 0
        for message in range(0, len(training_labels)):
            self.numOfSpams += training_labels[message] #the label of spam is 1 so adding all of the array gives the spam num
        self.numOfNonSpam = self.numOfMessages - self.numOfSpams
        self.totalNumOfWords = 0;
        for i in range(0,self.numOfFeatures):
            self.totalNumOfWords += self.FeaturePerLabel_0[i] + self.FeaturePerLabel_1[i]
        self.totalNumOfWordsinNonSpam = 0
        self.totalNumOfWordsinSpam = 0
        for i in range(0,self.numOfFeatures):
            self.totalNumOfWordsinSpam += self.FeaturePerLabel_1[i]
            self.totalNumOfWordsinNonSpam += self.FeaturePerLabel_0[i]
 
        stop = timeit.default_timer()
        print('training time: ', stop - start)
    
    #keeping numOfMessages, numOfFeatures, FeaturePerLabel_0, FeaturePerLabel_1, numOfSpams, numOfNonSpam, totalNumOfWords
        
    def probabilityOfLabel(self, label):
        assert ((label == 0) or (label == 1)), "label num can only be 0 or 1"
        if (label == 0):
            return self.numOfNonSpam/self.numOfMessages
        else:
            return self.numOfSpams/self.numOfMessages
        
    def probabilityOfFeature(self, featureNum):
        return (self.FeaturePerLabel_0[featureNum] + self.FeaturePerLabel_1[featureNum])/self.totalNumOfWords
        
    def probabilityOfFeatureGivenLabel(self, featureNum, label):
        assert self.probabilityOfLabel(label) != 0, 'probabilityOfLabel cannot be zero'
        if label == 0:           
            return (self.FeaturePerLabel_0[featureNum]/self.totalNumOfWordsinNonSpam)
        else:
            return (self.FeaturePerLabel_1[featureNum]/self.totalNumOfWordsinSpam)
                
    def probabilityOfLabelGivenFeature(self, featureNum, label):
        if self.probabilityOfFeature(featureNum) != 0:
            if label == 0:
                return self.FeaturePerLabel_0[featureNum]/(self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum])
            else:

                return self.FeaturePerLabel_1[featureNum]/(self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum])
        else:
            return 0            
        
    def jointProbability(featureNum, labelNum):
        return probabilityOfFeatureGivenLabel(featureNum,label)*self.probabilityOfLabel(label)   
        
        
    def predict(self,sample):
           # assert len(sample) == self.numOfFeatures, "sample has problems"
        prediction_label_0 = np.log(self.probabilityOfLabel(0))
        prediction_label_1 = np.log(self.probabilityOfLabel(1))
        for i in range(0, self.numOfFeatures):
            probabilityOfNonSpam = self.probabilityOfFeatureGivenLabel(i,0)
            probabilityOfSpam = self.probabilityOfFeatureGivenLabel(i,1)
            if sample[i] != 0:
                prediction_label_0 += sample[i]*np.log(probabilityOfNonSpam)
                prediction_label_1 += sample[i]*np.log(probabilityOfSpam)
    
        if prediction_label_0 > prediction_label_1:
            return 0
        elif prediction_label_0 == prediction_label_1:
            return 0
        else:
            return 1
        
    #keeping numOfMessages, numOfFeatures, FeaturePerLabel_0, FeaturePerLabel_1, numOfSpams, numOfNonSpam, totalNumOfWords
    
    def learnFromSample(self, sample,sampleLabel):
        assert len(sample) == self.numOfFeatures, "sample not valid"
        if sampleLabel == 0:
            self.numOfNonSpam += 1
            self.numOfMessages += 1
            for i in range(0,len(sample)):
                self.FeaturePerLabel_0[i] += sample[i]
                self.totalNumOfWordsinNonSpam += sample[i]
                self.totalNumOfWords += sample[i]
        else:
            self.numOfSpams += 1
            self.numOfMessages += 1
            for i in range(0,len(sample)):
                self.FeaturePerLabel_1[i] += sample[i]
                self.totalNumOfWordsinNonSpam += sample[i]
                self.totalNumOfWords += sample[i]     
        #todo mutual information   
                

In [2]:
def calc_accuracy(gt_y, pred_y):
    correct = 0
    for g_y, p_y in zip(gt_y, pred_y):
        if g_y == p_y: 
            correct += 1
            
    return (correct/float(len(gt_y))*100)

def confusion_matrix(correct_labels, predictions):
    confusion = [[0,0], [0, 0]]
    #tp fn; fp tn
    for l, p in zip(correct_labels, predictions):
        if l == p:
            if l == 0: #true negative
                confusion[1][1] += 1
            else:      #true positive
                confusion[0][0] += 1
        else: 
            if l == 0: #false positive
                confusion[1][0] += 1
            else: # false negative
                confusion[0][1] += 1
                
    return confusion

In [3]:
#get the data

#root = 'C:/Users/sezin/Desktop/SON SENE/CS464-Machine Learning/HW1/q3/'
#csv_path_training_data = os.path.join(root, 'sms_train_features.csv')
#csv_path_test_data = os.path.join(root, 'sms_test_features.csv')
#csv_path_training_label = os.path.join(root, 'sms_train_labels.csv')
#csv_path_test_label = os.path.join(root, 'sms_test_labels.csv')

csv_path_training_data =  'sms_train_features.csv'
csv_path_test_data = 'sms_test_features.csv'
csv_path_training_label = 'sms_train_labels.csv'
csv_path_test_label = 'sms_test_labels.csv'

def get_dataset(filename):
     with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        next(lines)
        dataset = list(lines)
        data = np.array([[float(row[col_i]) for col_i in range(1,len(row))] for row in dataset])
        #print(data)
        return data
    
train_features = get_dataset(csv_path_training_data)
test_features = get_dataset(csv_path_test_data)
train_labels = get_dataset(csv_path_training_label)
test_labels = get_dataset(csv_path_test_label)

naiveBayes1 = multinomialNaiveBayes(train_features,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes1.predict(test_features[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)


print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

training time:  27.919546399993123


C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/1874244885.py:82: RuntimeWarning: divide by zero encountered in log
  prediction_label_0 += sample[i]*np.log(probabilityOfNonSpam)
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/1874244885.py:83: RuntimeWarning: divide by zero encountered in log
  prediction_label_1 += sample[i]*np.log(probabilityOfSpam)


validation time:  29.817757899989374
94.3762781186094
[[99, 41], [14, 824]]


In [4]:


for i in range(0, len(test_labels)):
    naiveBayes1.learnFromSample(test_features[i],test_labels[i])

new_predictions = []
for j in range(0,len(test_labels)):
    prediction = naiveBayes1.predict(test_features[j])
    new_predictions.append(prediction)
    

C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/1874244885.py:82: RuntimeWarning: divide by zero encountered in log
  prediction_label_0 += sample[i]*np.log(probabilityOfNonSpam)
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/1874244885.py:83: RuntimeWarning: divide by zero encountered in log
  prediction_label_1 += sample[i]*np.log(probabilityOfSpam)


In [5]:
print(calc_accuracy(test_labels , new_predictions))
print(confusion_matrix(test_labels , new_predictions))

97.95501022494888
[[137, 3], [17, 821]]


In [6]:
class bernoulliNaiveBayes:
    
    def __init__(self, training_features, training_labels):
        start = timeit.default_timer()
        self.numOfMessages = len(training_labels)
        self.numOfFeatures = len(training_features[0])
        
        self.FeaturePerLabel_0 = [0]*self.numOfFeatures # given label 0, the feature x has been observed FeaturePerLabel_0[x] times
        self.FeaturePerLabel_1 = [0]*self.numOfFeatures # given label 1, the feature x has been observed FeaturePerLabel_1[x] times
        
        for featureNum in range(0,self.numOfFeatures):
            for row in range(0,len(training_labels)):
                if(training_features[row,featureNum] > 0):
                    if (training_labels[row] == 0):
                        self.FeaturePerLabel_0[featureNum] += 1
                    else:
                        self.FeaturePerLabel_1[featureNum] += 1
        
        self.numOfSpams = 0
        for message in range(0, len(training_labels)):
            self.numOfSpams += training_labels[message] #the label of spam is 1 so adding all of the array gives the spam num
        self.numOfNonSpam = self.numOfMessages - self.numOfSpams
        self.totalNumOfWords = 0;
        for i in range(0,self.numOfFeatures):
            self.totalNumOfWords += self.FeaturePerLabel_0[i] + self.FeaturePerLabel_1[i]
        self.totalNumOfWordsinNonSpam = 0
        self.totalNumOfWordsinSpam = 0
        for i in range(0,self.numOfFeatures):
            self.totalNumOfWordsinSpam += self.FeaturePerLabel_1[i]
            self.totalNumOfWordsinNonSpam += self.FeaturePerLabel_0[i]
        
        stop = timeit.default_timer()

        print('training time: ', stop - start)
    
    #keeping numOfMessages, numOfFeatures, FeaturePerLabel_0, FeaturePerLabel_1, numOfSpams, numOfNonSpam, totalNumOfWords
        
    def probabilityOfLabel(self, label):
        assert ((label == 0) or (label == 1)), "label num can only be 0 or 1"
        if (label == 0):
            return self.numOfNonSpam/self.numOfMessages
        else:
            return self.numOfSpams/self.numOfMessages
        
    def probabilityOfFeature(self, featureNum):
        return (self.FeaturePerLabel_0[featureNum] + self.FeaturePerLabel_1[featureNum])/self.totalNumOfWords
        
    def probabilityOfFeatureGivenLabel(self, featureNum, label):
        assert self.probabilityOfLabel(label) != 0, 'probabilityOfLabel cannot be zero'
        if label == 0:           
            return (self.FeaturePerLabel_0[featureNum]/self.totalNumOfWordsinNonSpam)
        else:
            return (self.FeaturePerLabel_1[featureNum]/self.totalNumOfWordsinSpam)
                
    def probabilityOfLabelGivenFeature(self, featureNum, label):
        if self.probabilityOfFeature(featureNum) != 0:
            if label == 0:
                return self.FeaturePerLabel_0[featureNum]/(self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum])
            else:

                return self.FeaturePerLabel_1[featureNum]/(self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum])
        else:
            return 0            
        
    def jointProbability(self,featureNum, labelNum):
        return self.probabilityOfFeatureGivenLabel(featureNum,labelNum)*self.probabilityOfLabel(labelNum)

        
    def predict(self,sample):
           # assert len(sample) == self.numOfFeatures, "sample has problems"
        prediction_label_0 = np.log(self.probabilityOfLabel(0))
        prediction_label_1 = np.log(self.probabilityOfLabel(1))
        for i in range(0, self.numOfFeatures):
            probabilityOfNonSpam = self.probabilityOfFeatureGivenLabel(i,0)
            probabilityOfSpam = self.probabilityOfFeatureGivenLabel(i,1)
            if sample[i] != 0:
                prediction_label_0 += np.log(probabilityOfNonSpam)
                prediction_label_1 += np.log(probabilityOfSpam)
    
        if prediction_label_0 > prediction_label_1:
            return 0
        elif prediction_label_0 == prediction_label_1:
            return 0
        else:
            return 1
        
    #keeping numOfMessages, numOfFeatures, FeaturePerLabel_0, FeaturePerLabel_1, numOfSpams, numOfNonSpam, totalNumOfWords
    
    def learnFromSample(self, sample,sampleLabel):
        assert len(sample) == self.numOfFeatures, "sample not valid"
        if sampleLabel == 0:
            self.numOfNonSpam += 1
            self.numOfMessages += 1
            for i in range(0,len(sample)):
                self.FeaturePerLabel_0[i] += 1
                self.totalNumOfWordsinNonSpam += 1 
                self.totalNumOfWords += 1
        else:
            self.numOfSpams += 1
            self.numOfMessages += 1
            for i in range(0,len(sample)):
                self.FeaturePerLabel_1[i] += 1
                self.totalNumOfWordsinNonSpam +=1
                self.totalNumOfWords += 1        
        #todo mutual information    
  
    def mutual_information(self,featureNum):
        if self.numOfMessages > 0 and ((self.FeaturePerLabel_0[featureNum] + self.FeaturePerLabel_1[featureNum]) * self.totalNumOfWordsinNonSpam)  > 0:
            entropy_of_featureNum00 = (self.FeaturePerLabel_0[featureNum]/self.numOfMessages)*np.log2( (self.numOfMessages*self.FeaturePerLabel_0[featureNum])/ (((self.FeaturePerLabel_0[featureNum] + self.FeaturePerLabel_1[featureNum]) * self.totalNumOfWordsinNonSpam)  ))
            entropy_of_featureNum01 = (self.FeaturePerLabel_1[featureNum]/self.numOfMessages)*np.log2((self.numOfMessages*self.FeaturePerLabel_1[featureNum])/ (((self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum]) * self.totalNumOfWordsinSpam)  ))                                                           
            return entropy_of_featureNum00 + entropy_of_featureNum01
        else:
            return 0
        
    def selectFeaturesUsingMutualInformation(self,featureNumToBeSelected): #returns new training data column numbers
    #open and train a new model after obtaining new training data
        assert featureNumToBeSelected < self.numOfFeatures
        mutualInfo = [0]*self.numOfFeatures
        #sorted(range(len(s)), key=lambda k: s[k])
        for i in range(0, self.numOfFeatures):
            mutualInfo[i] = self.mutual_information(i)
        sorted_mutualInfo_index = np.argsort(mutualInfo)
        
        selected_features = [0]*featureNumToBeSelected
        for i in range(0,featureNumToBeSelected):
            selected_features[i] = sorted_mutualInfo_index[i]
        selected_features.sort()
        return selected_features

In [7]:
naiveBayes2 = bernoulliNaiveBayes(train_features,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes2.predict(test_features[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))


training time:  5.352794100006577


C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:77: RuntimeWarning: divide by zero encountered in log
  prediction_label_0 += np.log(probabilityOfNonSpam)
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:78: RuntimeWarning: divide by zero encountered in log
  prediction_label_1 += np.log(probabilityOfSpam)


validation time:  23.974467499996535
94.3762781186094
[[99, 41], [14, 824]]


In [8]:

print(naiveBayes2.probabilityOfLabelGivenFeature(1,0))
print(naiveBayes2.probabilityOfLabelGivenFeature(1,0)+naiveBayes2.probabilityOfLabelGivenFeature(1,1))


0.5
1.0


In [9]:
select100 = naiveBayes2.selectFeaturesUsingMutualInformation(100)
select200 = naiveBayes2.selectFeaturesUsingMutualInformation(200)
select300 = naiveBayes2.selectFeaturesUsingMutualInformation(300)
select400 = naiveBayes2.selectFeaturesUsingMutualInformation(400)
select500 = naiveBayes2.selectFeaturesUsingMutualInformation(500)
select600 = naiveBayes2.selectFeaturesUsingMutualInformation(600)

def eliminate_feature(feature, x):
    if x == 0:
        return np.array([[float(row[col_i]) for col_i in range(1,len(row))] for row in feature])
    elif x == feature.shape[1]-1:
        return np.array([[float(row[col_i]) for col_i in range(0,len(row)-1)] for row in feature])
    elif x < feature.shape[1]-1:
        firstPart = np.array([[float(row[col_i]) for col_i in range(0,x)] for row in feature])
        secondPart = np.array([[float(row[col_i]) for col_i in range(x+1,len(row))] for row in feature])
        return np.concatenate((firstPart, secondPart), axis=1)
    else:
        return feature
    
def eliminate_except(feature, featuresToKeep):
    return np.array([[float(row[col_i]) for col_i in featuresToKeep] for row in feature])



C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:110: RuntimeWarning: divide by zero encountered in log2
  entropy_of_featureNum01 = (self.FeaturePerLabel_1[featureNum]/self.numOfMessages)*np.log2((self.numOfMessages*self.FeaturePerLabel_1[featureNum])/ (((self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum]) * self.totalNumOfWordsinSpam)  ))
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:110: RuntimeWarning: invalid value encountered in double_scalars
  entropy_of_featureNum01 = (self.FeaturePerLabel_1[featureNum]/self.numOfMessages)*np.log2((self.numOfMessages*self.FeaturePerLabel_1[featureNum])/ (((self.FeaturePerLabel_0[featureNum]+self.FeaturePerLabel_1[featureNum]) * self.totalNumOfWordsinSpam)  ))
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:109: RuntimeWarning: divide by zero encountered in log2
  entropy_of_featureNum00 = (self.FeaturePerLabel_0[featureNum]/self.numOfMessages)*np.log2( (self.numOfMessages*self.Fe

In [10]:
print('t1')
training_1_f = eliminate_except(train_features,select100)
print('t2')
training_2_f = eliminate_except(train_features,select200)
print('t3')
training_3_f = eliminate_except(train_features,select300)
print('t4')
training_4_f = eliminate_except(train_features,select400)
print('t5')
training_5_f = eliminate_except(train_features,select500)
print('t6')
training_6_f = eliminate_except(train_features,select600)
test_1_f = eliminate_except(test_features,select100)
test_2_f = eliminate_except(test_features,select200)
test_3_f = eliminate_except(test_features,select300)
test_4_f = eliminate_except(test_features,select400)
test_5_f = eliminate_except(test_features,select500)
test_6_f = eliminate_except(test_features,select600)

naiveBayes21 = bernoulliNaiveBayes(training_1_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes21.predict(test_1_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('100 features:')
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

naiveBayes22 = bernoulliNaiveBayes(training_2_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes22.predict(test_2_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('200 features:')
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

naiveBayes23 = bernoulliNaiveBayes(training_3_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes23.predict(test_3_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('300 features:')
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

naiveBayes24 = bernoulliNaiveBayes(training_4_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes24.predict(test_4_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('400 features:')
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

naiveBayes25 = bernoulliNaiveBayes(training_5_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes25.predict(test_5_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('500 features:')
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))

naiveBayes26 = bernoulliNaiveBayes(training_6_f,train_labels)
predictions = []
start = timeit.default_timer()
for j in range(0,len(test_labels)):
    prediction = naiveBayes26.predict(test_6_f[j])
    predictions.append(prediction)
stop = timeit.default_timer()
print('validation time: ', stop - start)
print('600 features:')    
print(calc_accuracy(test_labels , predictions))
print(confusion_matrix(test_labels , predictions))


t1
t2
t3
t4
t5
t6
training time:  0.12312740000197664
validation time:  0.7170031000277959
100 features:
91.8200408997955
[[85, 55], [25, 813]]
training time:  0.25000810000346974
validation time:  1.7196112999808975
200 features:
93.55828220858896
[[103, 37], [26, 812]]
training time:  0.35506130001158454
validation time:  2.5181722000124864
300 features:
93.96728016359918
[[108, 32], [27, 811]]
training time:  0.7607785000000149
validation time:  2.91989449999528
400 features:
94.98977505112475
[[115, 25], [24, 814]]
training time:  1.1247887999925297
validation time:  4.190703800006304
500 features:
94.98977505112475
[[116, 24], [25, 813]]
training time:  0.715164499997627


C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:77: RuntimeWarning: divide by zero encountered in log
  prediction_label_0 += np.log(probabilityOfNonSpam)
C:\Users\sezin\AppData\Local\Temp/ipykernel_20444/252096302.py:78: RuntimeWarning: divide by zero encountered in log
  prediction_label_1 += np.log(probabilityOfSpam)


validation time:  3.8271329999843147
600 features:
94.88752556237219
[[113, 27], [23, 815]]
